In [ ]:
%load_ext autoreload 
%autoreload 2

# 0. Loading dataset and libraries

**General note:** to make this notebook readable (and runnable), we present here the results, and not the whole process, of our endeavors. This notebook follows the logic of our [data story](https://epfl-ada.github.io/ada-2024-project-laambada/). For implementation details, please refer to scripts and notebooks found in `.\src`

## 0.1 Load libraries

In [44]:
import numpy as np
from src.scripts.load_and_save import load_data

## 0.2 Load data

This is a cleaned version of the BindingDB. It contains entries with the family of targets we chose.

In [ ]:
df = load_data()
df.head()

# 1. Data story contents

## 1.1 Kinases looking for love
This part serves to introduce the targets belonging to the TYROSINE-PROTEIN KINASE HOPSCOTCH family, further refered to as tyrosine kinase family.

Previously, we selected the targets belonging to the tyrosine kinase family to conduct our analyses. This choice was primarily motivated by a manageable amount of targets, accompanied by a decent amount of ligands per target, a requirment for ML-based processing.

In [46]:
from src.utils.embeddings_plots import  reduce_family

# Let's reduce the number of targets
df['Target Name Detailed'] = df['Target Name']
df['Target Name'] =  df['Target Name'].apply( reduce_family)

## 1.2 What is a good match?
This part serves to motivate our choice of metrics that can be used to characterize a binding.

### 1.2.1 Metrics
We start by exploring some metrics provided by the dataset.

In [ ]:
# Usefull metrics 
metric_percent = {}
for col in ['Ki (nM)', 'IC50 (nM)', 'Kd (nM)','kon (M-1-s-1)', 'koff (s-1)', 'EC50 (nM)', 'pH', 'Temp (C)']:
    metric_percent[col] = len(df[col].dropna())/len(df)


import plotly.express as px
x =list(metric_percent.keys())
y=list(metric_percent.values())

fig = px.bar(x=x, y=y)

fig.update_layout(xaxis_title='', yaxis_title='Fraction of Nan')

fig.show()

fig.write_html('src/data/figures/nan_fraction.html')

At this point, we drop `kon (M-1-s-1)`, `koff (s-1)`, `EC50 (nM)` columns, as there is a lot of missing data and these metrics by themselves do not bring enough information about the binding.

Both `IC50 (nM)` and `Ki (nM)` provide meaningful information. `Ki (nM)` is more interpretable for our project, as it doesn't depend on specific reaction conditions like `Ki (nM)` does. However, there is much more data for `IC50 (nM)`, so we'll see how we'll handle that. 

_Side note: BindingDB is drug discovery-oriented, so we expected to see more IC50 metrics_ 

As for `pH` and `Temp (C)`, we keep them both for later analyses. 

`Kd (nM)` column doesn't have enough data (even if we calculate it ourselves using `kon (M-1-s-1)` and `koff (s-1)`, so we remove it.

In [48]:
metrics = ['pKi', 'pIC50'] 
features = ['pH', 'Temp (C)']

In [49]:
for metric in metrics : 
    df['p' + metric] = np.where(
        df[metric] > 0,  # Only apply log10 to positive values
        -np.log10(df[metric] * 1e-9),  # Transform to molar and take -log10
        np.nan  # Assign NaN for zero or negative values
    )

Even if `IC50 (nM)` can be less meaningful due to its dependency on essay conditions, it is highly correlated to `Ki (nM)`. One potential explanation is a certain uniformity in experiment conducted on the same target. Therefore, it looks like we can extract meaningfull information about the inhibitory potential from `IC50 (nM)`. 

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

df_Ki_IC50 = df.dropna(subset=['pKi', 'pIC50'])

# Pearson Correlation
pearson_corr = df_Ki_IC50[['pKi', 'pIC50']].corr(method='pearson').iloc[0, 1]

# Linear Regression
reg = LinearRegression()
reg.fit(df_Ki_IC50['pIC50'].values.reshape(-1, 1), df_Ki_IC50['pKi'].values.reshape(-1, 1))
df_Ki_IC50['pred_pki'] = reg.predict(df_Ki_IC50['pIC50'].values.reshape(-1, 1))
linear_coef = reg.coef_[0][0].round(4)

# Line and Scatter Plot
fig1 = px.line(df_Ki_IC50, x='pIC50', y='pred_pki')
fig1.update_traces(line_color='red')
fig2 = px.scatter(df_Ki_IC50, x='pIC50', y='pKi', color='Target Name')
fig3 = go.Figure(data=fig1.data + fig2.data)

# Adding Annotations
fig3.add_annotation(
    x=df_Ki_IC50['pIC50'].max()-0.5,
    y=df_Ki_IC50['pKi'].min()+0.5,
    text=f"Pearson Corr: {pearson_corr:.2f}<br>Linear Coef: {linear_coef:.2f}",
    showarrow=False,
    align="right",
    font=dict(size=14, color="black")
)

# Layout updates
fig3.update_layout(
    xaxis_title="pIC50",
    yaxis_title="pKi"
)

# Show plot
fig3.show()
fig3.write_html('src/data/figures/pKi_pIC50.html')

### 1.2.1 Chemical caracterization
Now we move to the chemical characterization of ligands obtained through RDKit Descriptors.

In [52]:
properties = ['Ligand MW', 'logP']	

n_metrics = len(metrics)
n_properties = len(properties)

In [ ]:
df_ic50 = df.dropna(subset='IC50 (nM)')
df_ki = df.dropna(subset='Ki (nM)')

def get_metrics_df(df, metric):
    return df.dropna(subset=metric)

len(df_ic50)

In [ ]:
from src.utils.exploration_and_clean import plot_chemical_property_distributions

properties_colors = ['#EB89B5', '#330C73']
plot_chemical_property_distributions(df, metrics, chemical_properties=properties, properties_colors=properties_colors, filepath='src/data/figures/chemchar1.html', plot_metrics=True) 


In [ ]:
df.head()
functionnal_groups = ["Aliphatic OH", "Aromatic NH", "Ester", "Ether", "Amide",	"Ketone", "Benzene Ring"]
df_functionnal_groups = df[functionnal_groups]
df_functionnal_groups.head()

In [56]:
counts = df_functionnal_groups.sum()

In [57]:
fig = px.histogram(x=functionnal_groups, y=counts)
fig.update_layout(xaxis_title= '', yaxis_title='Count')
fig.write_html('src/data/figures/functional_groups.html')

In [ ]:
metric = metrics[0]
property = properties[0]
fig = px.scatter(get_metrics_df(df, metric), x=property, y=metric, marginal_x="histogram", marginal_y="histogram", color='Target Name')
fig.update_layout(height=600, width=1200)
fig.show()

In [ ]:
metric = metrics[1]
property = properties[0]
fig = px.scatter(get_metrics_df(df, metric), x=property, y=metric, marginal_x="histogram", marginal_y="histogram", color='Target Name')
fig.update_layout(height=600, width=1700)
#fig.add_hline(y=1e3, line_dash="dash",row=1, col=1)
fig.show()

In [ ]:
metric = metrics[0]
property = properties[1]
fig = px.scatter(get_metrics_df(df, metric), x=property, y=metric, marginal_x="histogram", marginal_y="histogram", color='Target Name')
fig.update_layout(height=600, width=1200)
fig.show()

In [ ]:
metric = metrics[1]
property = properties[1]
fig = px.scatter(get_metrics_df(df, metric), x=property, y=metric, marginal_x="histogram", marginal_y="histogram", color='Target Name')
fig.update_layout(height=600, width=1700)
fig.show()

## 1.3 Match made in heaven
This section explores various ways of embedding ligands.

In [62]:
from src.run_reduction import run_analysis

path_RD_df  = 'src/data/embeddings_RDKIT_descriptors.csv.zip'
path_Mol2vec_df = 'src/data/embeddings_Mol2Vec.csv.zip'
path_Morgan_df = 'src/data/embeddings_Morgan_Fingerprint.csv.zip'
path_full_df = 'src/data/embeddings_full.csv.zip'


### 1.3.1 RDKit Descriptors space
In an intuitive approach, we constructed our first "embedding space" by putting all RDKit Descriptors together. After some processing and dimensionality reduction, this is what we obtained.

In [ ]:
run_analysis(df, path_RD_df, do_umap=True)

In [ ]:
# Let's do more chemical characterization!
import random

features_of_interest = ["NumValenceElectrons", "ExactMolWt", "MolWt", "Chi0", "LabuteASA", "Kappa1", "HeavyAtomMolWt", "HeavyAtomCount", "Chi0v", "Chi1", "SPS", "FractionCSP3", "NumSaturatedRings", "NumAliphaticRings", "BCUT2D_CHGLO", "BCUT2D_CHGHI", "BCUT2D_LOGPLOW", "SlogP_VSA6", "SMR_VSA7", "NumAromaticCarbocycles"]
colors = ["#{:06x}".format(random.randint(0, 0xFFFFFF)) for _ in features_of_interest]

df_embeddings = load_data('src/data/embeddings_RDKIT_descriptors.csv.zip')
plot_chemical_property_distributions(df, metrics, chemical_properties=features_of_interest[:5], properties_colors=colors[:5], filepath='src/data/chemchar2.html', plot_metrics=False, df_embeddings=df_embeddings) 

### 1.3.2. [Mol2vec](https://pubs.acs.org/doi/10.1021/acs.jcim.7b00616) embedding space
In a quest for a more homogeneous embedding, we tried out Mol2vec. Mol2vec is an unsupervised machine learning approach to learn vector representations of molecular substructures. Compounds are encoded as vectors by summing the vectors of the individual substructures. The direction of substructure vectors created by Mol2vec provides information about chemically related substructures. 

In [ ]:
run_analysis(df, path_Mol2vec_df, do_umap=True)

### 1.3.3. Morgan fingerprints space
We next interrogated Morgan fingerprints, a type of molecular fingerprints. They are based on the idea of transforming local structural information of a molecule into a fixed-length bit string. This representation captures information about the molecular structure in a way that is useful for comparing molecules or identifying structural similarity.

In [ ]:
run_analysis(df, path_Morgan_df, do_umap=True)


### 1.3.4. Combined embeddings
Finally, we combine all three embeddings tested until that.

In [ ]:
run_analysis(df, path_full_df, do_umap=True)